This Notebook inserts the data into three SQLite tables according to db_schema.sql

In [10]:
import sqlite3, json, os

DB_PATH  = os.path.join('..','data','output','sql','book_reviews.db')
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)
SCHEMA   = os.path.join('..','schema','sql','db_schema.sql')
PHRASES  = os.path.join('..','data','processed','phrases.json')

print("Metadata input exists:", os.path.exists(SCHEMA))

Metadata input exists: True


In [11]:
# Connect & create tables
conn = sqlite3.connect(DB_PATH)
cur  = conn.cursor()
with open(SCHEMA,'r',encoding='utf-8') as f:
    cur.executescript(f.read())


In [12]:
# Load JSON data
with open(PHRASES,'r',encoding='utf-8') as f:
    data = json.load(f)

In [13]:
# Insert into tables
for entry in data:
    asin    = entry.get('asin','')
    title   = entry.get('title','')
    genre   = entry.get('genre','')
    price   = entry.get('price','')
    author  = entry.get('author','')
    if asin == "0006498442":
        print(genre)
    # Books table
    cur.execute(
        "INSERT OR IGNORE INTO books (asin,title, author, price,genre) VALUES (?,?,?,?,?)",
        (asin, title, author, price, genre)
    )

    # Reviews table
    cur.execute(
        "INSERT INTO reviews (asin,rating,review_text,summary) VALUES (?,?,?,?)",
        (asin, entry.get('rating'), entry.get('text',''), entry.get('summary',''))
    )
    review_id = cur.lastrowid

    # Sentiments table
    for p in entry.get('phrases', []):
        cur.execute(
            "INSERT INTO sentiments (review_id,phrase,polarity) VALUES (?,?,?)",
            (review_id, p['text'], p['polarity'])
        )

conn.commit()
conn.close()
print("Data insertion complete")


Data insertion complete


In [14]:
# Verify & Query Counts
import sqlite3

conn = sqlite3.connect('database/book_reviews.db')
for table in ('books','reviews','sentiments'):
    cnt = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {cnt} rows")
conn.close()


OperationalError: unable to open database file